# Expansão de Consultas

<h3>Autora: Dandara Sousa </h3>

Essa é a resolução da primeira parte do segundo lab da disciplina de Recuperação da Informação. Nesta atividade a ideia é exercitar a noção de expansão de consultas. Utilizando mais uma vez a coleção de dados de notícias os passos a serem executados são:
1. Escrever uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).
2. Escrever uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorne as top-3 palavras em ordem decrescente de frequência.
3. Expandir a consulta original com os termos retornados no passo 2 acima.
4. Fazer uma busca disjuntiva (OR) considerando a nova consulta.

Em seguida, escolhendo três termos de consulta, responder:
1. Quais os termos retornados para a expansão de cada consulta?
2. Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
3. Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
4. A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

In [1]:
import pandas as pd
import numpy as np
import nltk as nl
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps

Primeiro, vamos relembrar o conjunto de dados. Ele é formado por notícias do Estadão com as seguintes informações: o timestamp, o título da notícia, o subtítulo da notícias, o conteúdo da notícia, a url e, por último, o id.

In [2]:
df = pd.read_csv("estadao_noticias_eleicao.csv", encoding='utf-8')
df = df.replace(np.nan, '', regex = True)
content = df.titulo + " " + df.conteudo
content = content.fillna("")
df.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


<h4>Bigrama</h4>

*A função a seguir foi desenvolvida por Allan Sales e pode ser encontrada [aqui](https://github.com/allansales/information-retrieval/blob/master/Lab%203/coocurrence_matrix.ipynb).*

In [ ]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index